In [1]:
import warnings
warnings.filterwarnings('ignore')

# 교차검증과 그리드 서치
- 머신러닝을 사용할 때 모델의 정확도를 측정하기 위해 반드시 사용해야 하는 방법.    
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [2]:
import pandas as pd
wine = pd.read_csv("../Data/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# 검증 세트 추가

In [4]:
# Train 60% Valiadate 20% --검증 과정
# Test 20% -- 일반화 과정 

# 전체 세트 중 훈련세트와 테스트 세트를 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

train_input, test_input,train_target,test_target = train_test_split(
    data,target,test_size=0.2,random_state=42
)

In [5]:
# 훈련세트 중 훈련세트와 검증세트를 8:2의 기준으로 분리한다
sub_input, val_input, sub_target,val_target = train_test_split(
    train_input,train_target, test_size=0.2, random_state=42
)

In [6]:
# 훈련세트, 검증세트 , 테스트 세트의 크기 구하기
print("훈련세트 :",sub_input.shape)
print("검증세트 :",val_input.shape)
print("테스트세트 :",test_input.shape)

훈련세트 : (4157, 3)
검증세트 : (1040, 3)
테스트세트 : (1300, 3)


In [7]:
# 훈련세트와 검증세트를 결정트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)

print("Train score :",dt.score(sub_input,sub_target))
print("Valid score :",dt.score(val_input,val_target))

# 과대적합 문제 발생

Train score : 0.9971133028626413
Valid score : 0.864423076923077


---
# 교차검증

In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt,train_input,train_target)
scores

{'fit_time': array([0.00508022, 0.00473809, 0.00474691, 0.00480914, 0.00453186]),
 'score_time': array([0.0005188 , 0.00041294, 0.00037026, 0.00047302, 0.00033307]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [9]:
import numpy as np

np.mean(scores['test_score'])

0.855300214703487

---
# KFold : 분할기를 사용한 교차검증

In [10]:
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=5) # 기본 n_splits는 5
scores = cross_validate(dt,train_input,train_target,cv=splitter)
scores

{'fit_time': array([0.00477386, 0.00479794, 0.00498009, 0.00455403, 0.00438118]),
 'score_time': array([0.00047636, 0.000489  , 0.00044513, 0.000314  , 0.00036097]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [11]:
np.mean(scores['test_score'])

0.855300214703487

In [12]:
# KFold의 Fold를 10개로 나누어서 교차검증

splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42) # 기본 n_splits는 5
scores = cross_validate(dt,train_input,train_target,cv=splitter)
scores
# np.mean(scores['test_score'])

# shuffle => 과대적합 방지 

{'fit_time': array([0.00606513, 0.00561094, 0.00544572, 0.00505114, 0.00486779,
        0.00511479, 0.00504708, 0.00496006, 0.00502896, 0.0052197 ]),
 'score_time': array([0.00049686, 0.00031495, 0.00041819, 0.00023508, 0.00024009,
        0.00031209, 0.00023508, 0.00022984, 0.00025988, 0.00034928]),
 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
        0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}

In [13]:
splitter = StratifiedKFold(n_splits=10) # 기본 n_splits는 5
scores = cross_validate(dt,train_input,train_target,cv=splitter)
np.mean(scores['test_score'])

0.8616407292129834